In [1]:
import pandas as pd
import folium
import numpy as np
from geopy.distance import geodesic
from fuzzywuzzy import fuzz, process

C:\Users\Admin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv("C:/Users/Admin/Downloads/worldcities.csv")

# Recommend Country

In [17]:
def recommend_countries(origin_country, destination_countries):
    # Filter for capital cities
    df_capitals = df[df["capital"] == 1]

    # Group cities by country
    grouped = df_capitals.groupby("country")

    # Create a dictionary to store the capital cities for each country
    capitals = {}
    for country, group in grouped:
        capitals[country] = list(zip(group["lat"], group["lng"]))

    # Get the coordinates of the origin country's capital
    origin_coords = capitals[origin_country][0]

    # Create a list to store the recommended order of destination countries
    recommended_order = [origin_country]

    # Loop through the destination countries and find the closest one to the current country
    current_country = origin_country
    while len(recommended_order) < len(destination_countries)+1:
        distances = []
        for country in destination_countries:
            if country not in recommended_order:
                destination_coords = capitals[country][0]
                distance = geodesic(capitals[current_country][0], destination_coords).km
                distances.append(distance)
            else:
                distances.append(float("inf"))
        closest_country = destination_countries[distances.index(min(distances))]
        recommended_order.append(closest_country)
        current_country = closest_country

    # Calculate the distances between the origin and each destination country's capital
    distances = []
    for i in range(len(recommended_order)-1):
        origin_coords = capitals[recommended_order[i]][0]
        destination_coords = capitals[recommended_order[i+1]][0]
        distance = geodesic(origin_coords, destination_coords).km
        distances.append(distance)

    # Print the recommended order of destination countries with their destination number and distance from the previous country
    print("Recommended order of destination countries with their destination number and distance from the previous country:")
    print(f"1. {origin_country} (Distance: 0 km)")
    for i in range(1, len(recommended_order)):
        country = recommended_order[i]
        distance = distances[i-1]
        print(f"{i+1}. {country} (Distance from previous country: {distance} km)")
        
    # Draw the map
    m = folium.Map(location=capitals[recommended_order[0]][0], zoom_start=4)

    # Add markers for the origin and each destination country
    folium.Marker(location=capitals[recommended_order[0]][0], popup=origin_country, icon=folium.Icon(color="green")).add_to(m)
    for i in range(1, len(recommended_order)):
        country = recommended_order[i]
        coords = capitals[country][0]
        popup = f"{i}. {country}"
        icon_color = "red" if i == 1 else "blue"
        folium.Marker(location=coords, popup=popup, icon=folium.Icon(color=icon_color)).add_to(m)

        # Add a line connecting the markers
        prev_country = recommended_order[i-1]
        prev_coords = capitals[prev_country][0]
        folium.PolyLine([prev_coords, coords], color="gray").add_to(m)

    # Calculate the total distance
    total_distance = sum(distances)

    # Display the map
    return m

In [18]:
# define list of country names
countries = df['country'].drop_duplicates()


# Use fuzzywuzzy to match the user's input for the origin country with country names in dataset
origin_input = input("Enter the origin country: ")
origin_match = process.extractOne(origin_input, countries, scorer=fuzz.token_sort_ratio)

origin = origin_match[0]
# Use fuzzywuzzy to match the user's input for the destination countries with country names in dataset
destination_countries_str = input("Enter the destination countries (separated by commas): ")
destination_countries = [process.extractOne(c.strip(), countries, scorer=fuzz.token_sort_ratio)[0] 
                        for c in destination_countries_str.split(",")]
# Call the recommend_countries function with the matched origin and destination countries
recommend_countries(origin, destination_countries)

#Vietnam, Japan, South Korea, Thailand, Indonesia, Philippines, Singapore, Argentina, France, Germany, Brazil, San Tiago, Chile, Colombia, Peru

Enter the origin country: Puerto Rico
Enter the destination countries (separated by commas): Vietnam, Japan, South Korea, Thailand, Indonesia, Philippines, Singapore, Argentina, France, Germany
Recommended order of destination countries with their destination number and distance from the previous country:
1. Puerto Rico (Distance: 0 km)
2. Argentina (Distance from previous country: 5921.387405548339 km)
3. France (Distance from previous country: 11024.818699006724 km)
4. Germany (Distance from previous country: 879.6934626754954 km)
5. South Korea (Distance from previous country: 8149.045916435184 km)
6. Japan (Distance from previous country: 1154.0263990682192 km)
7. Philippines (Distance from previous country: 2991.477000332197 km)
8. Vietnam (Distance from previous country: 1752.8444993481605 km)
9. Thailand (Distance from previous country: 986.0266762211404 km)
10. Singapore (Distance from previous country: 1424.5315709610852 km)
11. Indonesia (Distance from previous country: 892.4

# Recommend City

In [4]:
def recommend_cities(previous_city, destination_cities):
    #Group cities by country and city
    grouped = df.groupby(["city"])

    #Create a dictionary to store the cities for each country
    cities = {}
    for city, group in grouped:
        cities[city] = list(zip(group["lat"], group["lng"]))
    
    # Get the coordinates of the origin country's capital
    origin_coords = cities[previous_city][0]
    
    # Create a list to store the recommended order of destination cities
    recommended_order = [previous_city]

    # Loop through the destination cities and find the closest one to the current city
    current_city = previous_city
    while len(recommended_order) < len(destination_cities)+1:
        distances = []
        for city in destination_cities:
            if city not in recommended_order:
                destination_coords = cities[city][0]
                distance = geodesic(cities[current_city][0], destination_coords).km
                distances.append(distance)
            else:
                distances.append(float("inf"))
        closest_city = destination_cities[distances.index(min(distances))]
        recommended_order.append(closest_city)
        current_city = closest_city

    # Calculate the distances between the origin and each destination country's capital
    distances = []
    for i in range(len(recommended_order)-1):
        origin_coords = cities[recommended_order[i]][0]
        destination_coords = cities[recommended_order[i+1]][0]
        distance = geodesic(origin_coords, destination_coords).km
        distances.append(distance)

    # Print the recommended order of destination countries with their destination number and distance from the previous country
    print("Recommended order of destination cities with their destination number and distance from the previous city:")
    print(f"1. {previous_city} (Distance: 0 km)")
    for i in range(1, len(recommended_order)):
        city = recommended_order[i]
        distance = distances[i-1]
        print(f"{i+1}. {city} (Distance from previous city: {distance} km)")
        
    # Draw the map
    m = folium.Map(location=cities[recommended_order[0]][0], zoom_start=4)

    # Add markers for the origin and each destination country
    folium.Marker(location=cities[recommended_order[0]][0], popup=previous_city, icon=folium.Icon(color="green")).add_to(m)
    for i in range(1, len(recommended_order)):
        city = recommended_order[i]
        coords = cities[city][0]
        popup = f"{i}. {city}"
        icon_color = "red" if i == 1 else "blue"
        folium.Marker(location=coords, popup=popup, icon=folium.Icon(color=icon_color)).add_to(m)

        # Add a line connecting the markers
        prev_city = recommended_order[i-1]
        prev_coords = cities[prev_city][0]
        folium.PolyLine([prev_coords, coords], color="gray").add_to(m)

    # Calculate the total distance
    total_distance = sum(distances)

    # Display the map
    return m

In [5]:
# define list of country names
countries = df['country'].drop_duplicates()


# Use fuzzywuzzy to match the user's input for the previous country with country names in dataset
previous_input = input("Enter the previous country: ")
previous_match = process.extractOne(previous_input, countries, scorer=fuzz.token_sort_ratio)

previous_country = previous_match[0]

# define list of city names
prev_cities = df.loc[df['country'] == previous_country, 'city']

# Use fuzzywuzzy to match the user's input for the previous city with city names in dataset
previous_input = input("Enter the previous city: ")
previous_match = process.extractOne(previous_input, prev_cities, scorer=fuzz.token_sort_ratio)

previous_city = previous_match[0]

#Destination country
# define list of country names
# Use fuzzywuzzy to match the user's input for the destination country with country names in dataset
dest_input = input("Enter the destination country: ")
dest_match = process.extractOne(dest_input, countries, scorer=fuzz.token_sort_ratio)

dest_country = dest_match[0]
#List of city names of destination country
dest_cities = df.loc[df['country'] == dest_country, 'city']
# Use fuzzywuzzy to match the user's input for the destination countries with country names in dataset
destination_cities_str = input("Enter the destination cities (separated by commas): ")
destination_cities = [process.extractOne(c.strip(), dest_cities, scorer=fuzz.token_sort_ratio)[0]
                     for c in destination_cities_str.split(",")]
# Call the recommend_cities function with the matched previous and destination cities
recommend_cities(previous_city, destination_cities)

#ha noi, hue, hai phong, da nang, tam ky, quang ngai, Can tho, bien hoa, thanh hoa, nha trang, nam dinh, vung tau, yen bai, cao bang, bac kan, chi linh

Enter the previous country: Japan
Enter the previous city: Tokyo
Enter the destination country: Viet Nam
Enter the destination cities (separated by commas): ha noi, hue, hai phong, da nang, tam ky, quang ngai, Can tho, bien hoa, thanh hoa, nha trang, nam dinh, vung tau, yen bai, cao bang, bac kan, chi linh
Recommended order of destination cities with their destination number and distance from the previous city:
1. Tokyo (Distance: 0 km)
2. Cao Bang (Distance from previous city: 3535.293613926623 km)
3. Bac Kan (Distance from previous city: 73.51161600188377 km)
4. Yen Bai (Distance from previous city: 110.79790210685728 km)
5. Ha Noi (Distance from previous city: 126.57310933621082 km)
6. Chi Linh (Distance from previous city: 56.15935226897454 km)
7. Hai Phong (Distance from previous city: 43.10303172649375 km)
8. Nam Dinh (Distance from previous city: 72.9006269916013 km)
9. Thanh Hoa (Distance from previous city: 79.22939066023174 km)
10. Hue (Distance from previous city: 416.064750